# Maximum TASK LOOP Test 

# Test Description
This script will test the functionality of Task loop.
As per datasheet maximum value of Task loop is 1M.
Here task is created having sine wave of 1 cycle and execute it in a loop of 1M.
Oscilloscope will measure the count of sine wave cycles.
Test is pass if no of cycle of sine wave measured is 1M.


# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable

# Hardware Connection
    Proteus Module    --->     Oscilloscope
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4


# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release - Aurora

In [1]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [2]:
# Connect to instrument
connection = 'LAN' #input ("Do you wish to connect via LAN/PXI/LOCAL/USB ")
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    inst=connect("2",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2


Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [3]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
# Get the DAC mode (8 bits or 16 bits)

resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8
   
print('DAC {0} bits'.format(dac_mode))

Model: P9484M 
DAC 16 bits


In [4]:
# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
#proteus.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
num_channels = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + num_channels)
#num_channels = int(resp)

0, no error
Number of channels: 4


In [5]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

for i in range(4):
    scope.write(':CHANnel{}:DISPlay OFF'.format(i+1)) 
    scope.write(':MEASure:CLEar')


*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [6]:
InitialFreqSampleRate = 1000
MaximumFreqSampleRate = 9000
FreqSampleRateJump = 1000

# print( 'Internal trigger Test for sampling rate {}MHz to {}MHz in jumps of {}MHz'.format(InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump)) 
for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):
    inst.send_scpi_cmd(':FREQ:RAST {} MHz'.format(int(FreqSampleRate)))
    FreqSampleRate = int(FreqSampleRate)
    
    print('\n***Test begin for sampling rate {}MHz ***'.format(FreqSampleRate))
        
    if FreqSampleRate > 2500:
        channel_select = [1,3]
    else:
        channel_select = [1,2,3,4]
    
    for in_channelselect in range(len(channel_select)):
        # Select Channel & Clear it for new operation
        inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
        # Several initializations ..
        inst.send_scpi_cmd('*CLS; *RST')
        #proteus.send_scpi_cmd(':INST:CHAN 2')
        inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
        
        print('\n\t***Test Start for sampling rate {0}MHz for channel {1} ***'.format(FreqSampleRate,channel_select[in_channelselect]))  
        scope.write('*RST')
        

        
        # Build sine wave on channel 1
        segnum = 1
        seglen = 1024
        ncycles1 = 1
        # cyclelen = seglen
        cyclelen = (seglen/ncycles1)
        waves = [ None for _ in range(3)]
        # sin wave:
        x = np.linspace(start=0, stop=2 * np.pi * ncycles1, num=seglen, endpoint=False)
        y = (np.sin(x) + 1.0) * half_dac
        y = np.round(y)
        y = np.clip(y, 0, max_dac)
        waves[0] = y.astype(wpt_type)
#         plt.plot(x,y)
#         print("************* WAVEFORM*****************")
        # Download sine wave to channel 1
        for ii in range(1):
            ichan = ii
            channb = ichan + 1
            segnum = ii +1
        wav = waves[0]
#         print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
        # Select channel
        inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))    
        # Define segment
        inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
        # Select the segment
        inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
    #     proteus.send_scpi_cmd(':VOLT:OFFS 0.2')
        inst.send_scpi_cmd(':VOLT 0.5')
        # Increase the timeout before writing binary-data:
        inst.timeout = 30000    
        # Select the segment
        cmd = ':TRAC:SEL {0}'.format(segnum)
        inst.send_scpi_cmd(cmd)        
        # Send the binary-data:
        inst.write_binary_data(':TRAC:DATA', wav)
        resp = inst.send_scpi_query(':SYST:ERR?')
        resp = resp.rstrip()
        if not resp.startswith('0'):
            print('ERROR: "{}" after writing binary values'.format(resp))
        # Play the specified segment at the selected channel:
        cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
        inst.send_scpi_cmd(cmd)
        # Turn on the output of the selected channel:
        inst.send_scpi_cmd(':OUTP ON')    
        resp = inst.send_scpi_query(':SYST:ERR?')
#         print(resp)
#         print()
        
        
        
        inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
        inst.send_scpi_cmd(':INIT:CONT OFF')
        inst.send_scpi_cmd(':TRIG:SOUR:ENAB CPU')
        inst.send_scpi_cmd(':TRIG:STAT ON')

        
        
            
        taskloop = 1000000
        scope.write(':CHANnel{}:DISPlay ON'.format(channel_select[in_channelselect]))
        
        inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format(1))
        inst.send_scpi_cmd(':TASK:COMP:ENAB CPU') 
        inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(1))                  # play segment 2 SQP
        inst.send_scpi_cmd(':TASK:COMP:LOOP {0}'.format(taskloop))
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format(2))                 # the next task is 6
#         print('Task no {} is created successfully'.format(1))

#         inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format(2))
#         inst.send_scpi_cmd(':TASK:COMP:ENAB CPU') 
#         inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(1))                  # play segment 2 SQP
#         print('Task no {} is created successfully'.format(2))

                  
        inst.send_scpi_cmd(':TASK:COMP:WRIT')
        inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
        inst.send_scpi_cmd(':FUNC:MODE TASK')
        inst.send_scpi_cmd(':OUTP ON')
              
        scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
        scope.write(':CHANnel{}:SCALe 200E-3'.format(channel_select[in_channelselect]))
        scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
        scope.write(':TIMebase:SCALe 121E-3')
        scope.write(':TIMebase:POSition 482.4E-3')
        scope.write(':TRIGger:SWEep TRIGgered')
        scope.write(':TRIGger:MODE EDGE')
        scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(channel_select[in_channelselect]))
        scope.write(':TRIGger:LEVel CHANnel{},156e-3'.format(channel_select[in_channelselect]))
        
        time.sleep(2)
        
        inst.send_scpi_cmd('*TRG')
        
        time.sleep(2)
        
        
        
        
        scope.write(':MEASure:PPULses CHANnel{}'.format(channel_select[in_channelselect]))
        time.sleep(3)
        scope.write(':MEASure:RESults?')
        result = scope.read()
#         print(result)
        pulse_count = float(result.split(',')[2])
#         print(pulse_count)
        
        if(pulse_count==1000000):
            print('\t***Test Pass for sampling rate {0}MHz for channel {1} ***'.format(FreqSampleRate,channel_select[in_channelselect]))
        else:
            print('\t***Test Fail for sampling rate {0}MHz for channel {1} ***'.format(FreqSampleRate,channel_select[in_channelselect]))
            
        
        time.sleep(2)
     
    print('***Test completed for sampling rate {}MHz ***'.format(FreqSampleRate))
        
print('\n\n***Test completed for all sampling rate ***')        
# Close the connection to the instrument
scope.close()
resourceManager.close()
#disconnect  
# dis = input("Press enter to disconnect")
disconnect()
print("Device is Disconnected")    
        
  



***Test begin for sampling rate 1000MHz ***

	***Test Start for sampling rate 1000MHz for channel 1 ***
	***Test Pass for sampling rate 1000MHz for channel 1 ***

	***Test Start for sampling rate 1000MHz for channel 2 ***
	***Test Pass for sampling rate 1000MHz for channel 2 ***

	***Test Start for sampling rate 1000MHz for channel 3 ***
	***Test Pass for sampling rate 1000MHz for channel 3 ***

	***Test Start for sampling rate 1000MHz for channel 4 ***
	***Test Pass for sampling rate 1000MHz for channel 4 ***
***Test completed for sampling rate 1000MHz ***

***Test begin for sampling rate 2000MHz ***

	***Test Start for sampling rate 2000MHz for channel 1 ***
	***Test Pass for sampling rate 2000MHz for channel 1 ***

	***Test Start for sampling rate 2000MHz for channel 2 ***
	***Test Pass for sampling rate 2000MHz for channel 2 ***

	***Test Start for sampling rate 2000MHz for channel 3 ***
	***Test Pass for sampling rate 2000MHz for channel 3 ***

	***Test Start for sampling rate 20